# NIKE CONSOLIDATION
### Cre: Mr. Dang Tran

In [1]:
from datetime import datetime
import time
start_time = time.time()

In [2]:
import pandas as pd
import pyodbc
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Import table

In [4]:
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=ACFC-L-DANGNIKE\SQLEXPRESS;'
    'Database=NIKE;'
    'Trusted_Connection=yes;')

stock = pd.read_sql_query(
    """DECLARE @LD DATE = (SELECT MAX(CREATED_DATE) FROM SALES_DATA);
    DECLARE @FD DATE = DATEADD(WEEK,-8,@LD );

    WITH 
    --executing sales performance, comparing sell through
        INV AS -- Get total stock, group by store and sku
            (SELECT STORE_CODE, DESCRIPTION1 SKU, SUM(ASN_PENDING + CLOSING) STOCK_QTY
            FROM STOCK WHERE STORE_CODE LIKE '3%'
            GROUP BY STORE_CODE, DESCRIPTION1
            HAVING SUM(ASN_PENDING + CLOSING) > 0),

        SHP AS -- get Onstore date of each SKU in each Store
            (SELECT [STORE_CODE], SKU, MIN([Requested Delivery Date ]) OSD
            FROM STORE_SHIPIN S LEFT JOIN MASTER_UPC U ON S.[Product Code] = U.UPC
            WHERE [Requested Delivery Date ] BETWEEN @FD and @ld AND STORE_CODE LIKE '3%'
            GROUP BY [STORE_CODE], SKU),

        SAL AS -- get sold out qty from 8 nearest weeks
            (SELECT STORE_CODE, STYLE SKU, MIN(CREATED_DATE) FSD, SUM(QTY) SOLD_QTY
            FROM SALES_DATA WHERE CREATED_DATE BETWEEN @FD AND @LD
            group by STORE_CODE, STYLE),

        GRP AS --combine 3 tables above
            (SELECT
                M.SKU, DIV, GENDER, CAT, AGE, DES, SRP, AREA, S.[STORE CODE] STORE_CODE, [SHORT NAME],
                STDEV(
                    ISNULL(SOLD_QTY,0) / (ISNULL(SOLD_QTY,0) + ISNULL(STOCK_QTY,0)) / --Sell through
                    DATEDIFF(WEEK,ISNULL(IIF(FSD<OSD OR OSD IS NULL,FSD,OSD),@FD),@LD) --Weeks on store
                    ) OVER (PARTITION BY M.SKU, AREA) STDEV_ST,
                ROW_NUMBER() OVER(
                    PARTITION BY M.SKU, AREA ORDER BY ISNULL(SOLD_QTY,0) /
                    DATEDIFF(WEEK,ISNULL(IIF(FSD<OSD OR OSD IS NULL,FSD,OSD),@FD),@LD) DESC
                    ) STORE_RANK
            FROM 
                MASTER_FILE M CROSS JOIN MASTER_STORE S
                LEFT JOIN SAL ON M.SKU = SAL.SKU AND S.[STORE CODE] = SAL.STORE_CODE
                LEFT JOIN INV ON M.SKU = INV.SKU AND S.[STORE CODE] = INV.STORE_CODE
                LEFT JOIN SHP ON M.SKU = SHP.SKU AND S.[STORE CODE] = SHP.STORE_CODE   
            WHERE 
                STOCK_QTY > 0 AND
                ISNULL(SOLD_QTY,0) + ISNULL(STOCK_QTY,0) > 0 AND
                DATEDIFF(WEEK,ISNULL(IIF(FSD<OSD OR OSD IS NULL,FSD,OSD),@FD),@LD) >= 4),

    --Cleaning stock list
        RAW_INV AS --Raw stock from table stock
            (SELECT 
                STORE_CODE,
                DESCRIPTION1 SKU,
                SIZ,
                (ASN_PENDING+CLOSING) QTY
            FROM STOCK
            WHERE STORE_CODE LIKE '3%' AND (ASN_PENDING+CLOSING) > 0),

        PRD_LST AS --get all available size of each SKU
            (SELECT
                STORE_CODE, A.DESCRIPTION1 SKU, SIZ SIZE
            FROM
                (SELECT DISTINCT STORE_CODE, DESCRIPTION1 FROM STOCK WHERE ASN_PENDING+CLOSING>0) A
                LEFT JOIN
                (SELECT DISTINCT DESCRIPTION1, SIZ FROM STOCK WHERE ASN_PENDING+CLOSING>0) B
                ON A.DESCRIPTION1 = B.DESCRIPTION1
            WHERE STORE_CODE LIKE '3%'),

        TTL_INV AS --Combine full stock list
            (SELECT
                PRD_LST.STORE_CODE, PRD_LST.SKU, PRD_LST.SIZE,
                ISNULL(QTY,0) QUANTITY
            FROM 
                PRD_LST LEFT JOIN RAW_INV ON 
                    PRD_LST.STORE_CODE = RAW_INV.STORE_CODE AND 
                    PRD_LST.SKU = RAW_INV.SKU AND
                    PRD_LST.SIZE = RAW_INV.SIZ)

    --Combining ttl stock list and sales performance
    SELECT 
        TTL_INV.SKU, SIZE, DIV, GENDER, CAT, AGE, DES, SRP, 
        AREA, TTL_INV.STORE_CODE, [SHORT NAME], 
        QUANTITY STOCK_BEFORE, QUANTITY STOCK_AFTER, STORE_RANK
    FROM 
        TTL_INV 
        INNER JOIN GRP ON
            TTL_INV.STORE_CODE = GRP.STORE_CODE AND
            TTL_INV.SKU = GRP.SKU AND
            GRP.STDEV_ST >= 0.05
    ORDER BY SKU, AREA, STORE_RANK, STORE_CODE, SIZE;""",
    conn)

stock['STORE_IN'] = np.nan
stock.head()

,SKU,SIZE,DIV,GENDER,CAT,AGE,DES,SRP,AREA,STORE_CODE,SHORT NAME,STOCK_BEFORE,STOCK_AFTER,STORE_RANK,STORE_IN
0,311046-021,10,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,3.0,3.0,1,NaN
1,311046-021,10.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,1.0,1.0,1,NaN
2,311046-021,11,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,0.0,1,NaN
3,311046-021,7,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,0.0,1,NaN
4,311046-021,7.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,0.0,1,NaN


In [5]:
# Iterate over each SKU
for sku in stock['SKU'].drop_duplicates():
    
    # with every single SKU, iterate over each AREA
    for area in stock[stock['SKU'] == sku]['AREA'].drop_duplicates():
        
        #with every single AREA, iterate over store_code
        for store_code_in, rank_in in stock[(stock['SKU'] == sku) & (stock['AREA'] == area)][['STORE_CODE','STORE_RANK']].drop_duplicates().itertuples(index=False):
            
            #if this store alreay tranferred out, skip
            if len(stock[(stock['SKU'] == sku) & (stock['STORE_CODE'] == store_code_in) & (stock['STORE_IN'].notnull())]) >0:
                continue

            other_stores = stock[
                (stock['SKU'] == sku) & 
                (stock['STORE_CODE'] != store_code_in) & 
                (stock['AREA'] == area) &
                (stock['STORE_RANK'] > rank_in) & 
                (stock['STORE_IN'].isnull())
            ]
            
            for store_code_out in other_stores['STORE_CODE'][::-1].drop_duplicates():
                
                store_in = stock[(stock['SKU'] == sku) & (stock['STORE_CODE'] == store_code_in)]
                
                store_out = stock[(stock['SKU'] == sku) & (stock['STORE_CODE'] == store_code_out)]
                
                merge = store_in.append(store_out).groupby('SIZE',as_index=False)['STOCK_AFTER'].sum()
                
                if (merge['STOCK_AFTER'].max() <=6) & (merge[merge.STOCK_AFTER > 0]['STOCK_AFTER'].mean() <=3):
                    stock.loc[
                        (stock['SKU'] == sku) & (stock['STORE_CODE'] == store_code_in),
                        'STOCK_AFTER'
                    ] = merge.STOCK_AFTER.values.tolist()

                    stock.loc[
                        (stock['SKU'] == sku) & (stock['STORE_CODE'] == store_code_out),
                        ['STOCK_AFTER','STORE_IN']
                    ] = [0,store_code_in]
                    

In [6]:
stock

,SKU,SIZE,DIV,GENDER,CAT,AGE,DES,SRP,AREA,STORE_CODE,SHORT NAME,STOCK_BEFORE,STOCK_AFTER,STORE_RANK,STORE_IN
0,311046-021,10,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,3.0,4.0,1,NaN
1,311046-021,10.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,1.0,2.0,1,NaN
2,311046-021,11,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,0.0,1,NaN
3,311046-021,7,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,1.0,1,NaN
4,311046-021,7.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,1.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13430,SX7803-100,M,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,359,SVH,0.0,0.0,2,330
13431,SX7803-100,XL,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,359,SVH,0.0,0.0,2,330
13432,SX7803-100,L,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,344,LM81,0.0,0.0,3,330
13433,SX7803-100,M,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,344,LM81,0.0,0.0,3,330


In [10]:
stock['TRANSFER_OUT_QTY']=np.where(stock.STOCK_AFTER < stock.STOCK_BEFORE,stock.STOCK_BEFORE-stock.STOCK_AFTER,0)
stock = stock[(stock.STOCK_BEFORE > 0) | (stock.STOCK_AFTER > 0)].reset_index()
stock

,index,SKU,SIZE,DIV,GENDER,CAT,AGE,DES,SRP,AREA,STORE_CODE,SHORT NAME,STOCK_BEFORE,STOCK_AFTER,STORE_RANK,STORE_IN,TRANSFER_OUT_QTY
0,0,311046-021,10,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,3.0,4.0,1,NaN,0.0
1,1,311046-021,10.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,1.0,2.0,1,NaN,0.0
2,3,311046-021,7,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,1.0,1,NaN,0.0
3,4,311046-021,7.5,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,0.0,1.0,1,NaN,0.0
4,5,311046-021,8,FW,MALE,JOR,ADULT,AIR JORDAN DUB ZERO,5390000.0,SOUTH,351,SGC,3.0,4.0,1,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8545,13423,SX7803-100,L,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,MIDDLE,355,NTRA,1.0,0.0,2,367,1.0
8546,13426,SX7803-100,L,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,330,LVS,2.0,3.0,1,NaN,0.0
8547,13428,SX7803-100,XL,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,330,LVS,0.0,1.0,1,NaN,0.0
8548,13429,SX7803-100,L,EQ,MALE,ATR,ADULT,U NK EVERYDAY LTWT LOW 3PR,390000.0,SOUTH,359,SVH,1.0,0.0,2,330,1.0


In [11]:
file_name = r'D:\OneDrive\ACFC\NIKE - Documents\From G Suite Drive\5. MERCHANDISING TEAM\CONSOLIDATE\WKLY CONSOLIDATE\WKLY CONSOLIDATE '+ datetime.today().strftime('%Y%m%d') + '.xlsx'
stock.to_excel(file_name)

In [9]:
end_time = time.time()
print(end_time - start_time)

552.2557814121246
